In [1]:
from scipy.spatial.distance import pdist, squareform
from ripser import ripser
from itertools import product, starmap, islice
from ringity.classes import Dgm, DgmPt
from random import randint

import random
import scipy
import ringity
import numpy as np
import os
import time
import networkx as nx

In [ ]:
N = 5
M = 10
A = [(min(a,b), max(a,b)) for (a,b) in np.random.randint(0,100, size=(N,2))]
B = [(min(a,b), max(a,b)) for (a,b) in np.random.randint(0,100, size=(M,2))]

In [3]:
dgm_A = ringity.classes.Dgm(A)
dgm_B = ringity.classes.Dgm(B)

In [4]:
dgm = dgm_A + dgm_B

In [10]:
dgm.score

0.30888061523437527

In [9]:
ringity.classes.score(iter(dgm.sequence))

0.30888061523437527

In [7]:
ringity.classes.score([1,6,10])

0.675

In [8]:
%time
sorted(np.random.uniform(size=10**6))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs


[2.343679837202295e-06,
 2.403461171862631e-06,
 3.985107907444885e-06,
 5.253347384992502e-06,
 6.4434202123875295e-06,
 6.611511091381139e-06,
 7.410488915615687e-06,
 8.113729984104978e-06,
 8.139826549791174e-06,
 9.07909200320578e-06,
 9.422988639706809e-06,
 9.56144585206875e-06,
 1.0259413850355337e-05,
 1.0399503562963375e-05,
 1.1981231418545235e-05,
 1.2165499672756042e-05,
 1.2313960652265976e-05,
 1.4982235025051693e-05,
 1.4996149553692106e-05,
 1.5911526769052386e-05,
 1.641604219004833e-05,
 1.8021588776950814e-05,
 2.0043386145918163e-05,
 2.0390041214013443e-05,
 2.096541925333817e-05,
 2.0984903091325613e-05,
 2.1416490658565657e-05,
 2.2359188943665842e-05,
 2.3427369357564842e-05,
 2.520192322885695e-05,
 2.5326719775398132e-05,
 2.5784471592738e-05,
 2.6074861210645395e-05,
 2.622536337404391e-05,
 2.7937963381630126e-05,
 2.8752537027165737e-05,
 2.9020356936193537e-05,
 2.9073684361446972e-05,
 2.9146783139366583e-05,
 3.0311636035840195e-05,
 3.084031946087862e-

In [12]:
dgm

Dgm([(100, 200), (18, 80), (18, 79), (10, 68), (26, 83), (18, 73), (48, 98), (33, 82), (23, 71), (47, 92), (29, 68), (35, 63), (31, 58), (35, 55), (53, 63), (17, 20), (37, 37), (0, 0)])

In [29]:
%%timeit
list(zip(dgm.births, dgm.deaths))

6.83 µs ± 812 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [30]:
%%timeit
[[k.birth, k.death] for k in dgm]

9.31 µs ± 152 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [38]:
np.array(list(zip(dgm.births, dgm.deaths))) == np.array([[k.birth, k.death] for k in dgm])

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True]])

In [40]:
np.array(list(zip(dgm.births, dgm.deaths)))

array([[28, 95],
       [17, 80],
       [17, 80],
       [27, 82],
       [ 8, 52],
       [ 5, 46],
       [21, 62],
       [ 0, 34],
       [26, 51],
       [34, 58],
       [54, 78],
       [ 2, 18],
       [12, 20],
       [81, 85],
       [65, 68]])

In [51]:
pt1 = next(iter(dgm))
pt2 = next(islice(iter(dgm), 1,2))

In [11]:
sorted(np.random.uniform(size=2))

[0.12160123787044563, 0.76685487032098]

In [58]:
ringity.classes.DgmPt(*np.random.uniform(2,7,size=2))

(3.1059578659722167, 5.905193602055771)

In [69]:
number = random.random()
dgm = ringity.classes.Dgm([(0,number) for _ in range(100)])

In [70]:
dgm.score

1.7763568394002505e-15

In [71]:
a, b = np.random.uniform(2,7,size=2)

In [72]:
a

6.803158271598914

In [48]:
signal = random.random()
noise = [0]*random.randint(0,10)


noise_ratio2, noise_ratio1 = (0,0.123)

noise1 = signal*noise_ratio1
noise2 = signal*noise_ratio2

sequence = noise + [signal] + \
           noise + [noise1] + \
           noise + [noise2] + \
           noise

In [49]:
sequence = [signal, noise1, noise2]

In [50]:
ringity.classes.score(sequence)

0.9385

In [51]:
1-noise_ratio1/2 - noise_ratio2/4

0.9385

In [2]:
RINGITY_PATH = os.path.dirname(ringity.__file__)

In [3]:
G = nx.read_edgelist(f"{RINGITY_PATH}/test_data/lipid_coexp_network.csv",
                            nodetype=int)
G = nx.convert_node_labels_to_integers(G)
dgm1 = ringity.core.diagram(G)
dgm2 = ringity.classes.load_dgm(f"{RINGITY_PATH}/test_data/lipid_coexp_dgm.csv")

In [5]:
dgm1.save(f"{RINGITY_PATH}/test_data/test.txt", fmt='%.7f')

In [6]:
? np.savetxt

In [6]:
dgm1.score

0.7669806588679011